In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import hdbscan
from itertools import product
from time import time
import umap
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# 读取结果
df = pd.read_csv("umap_hdbscan_results_fashion.csv")

In [4]:
df.head(5)

,nn,md,nc,umap_metric,mcs,ms,eps,hdb_metric,Clusters,Silhouette,NoiseRatio,Score,TimeSec
0,50,0.1,4,euclidean,50,50,0.0,euclidean,2,0.705050,0.006777,0.698274,0.519
1,50,0.1,4,euclidean,50,50,0.1,euclidean,2,0.705050,0.006777,0.698274,0.512
2,50,0.1,4,euclidean,20,50,0.0,euclidean,2,0.705050,0.006777,0.698274,0.530
3,50,0.1,4,euclidean,20,50,0.1,euclidean,2,0.705050,0.006777,0.698274,0.512
4,10,0.0,8,euclidean,20,5,0.0,euclidean,72,0.671414,0.115629,0.555785,0.446


In [5]:
# 筛选条件：簇数适中 + 噪声不过大
df_filtered = df[
    (df["Clusters"] >= 10) & 
    (df["Clusters"] <= 30) & 
    (df["NoiseRatio"] <= 0.3)
]

# 按 Score 排序，取 Top 10
df_top = df_filtered.sort_values(by="Score", ascending=False).head(10)

In [7]:
df_filtered= df[
    (df["Clusters"] >= 10)
]

In [10]:
# 按 Score 排序，取 Top 10
df_top = df_filtered.sort_values(by="Silhouette", ascending=False).head(10)

In [11]:
df_top

,nn,md,nc,umap_metric,mcs,ms,eps,hdb_metric,Clusters,Silhouette,NoiseRatio,Score,TimeSec
19,10,0.0,4,euclidean,20,20,0.0,euclidean,58,0.733580,0.218340,0.515240,0.341
18,10,0.0,4,euclidean,20,20,0.1,euclidean,58,0.733580,0.218340,0.515240,0.366
126,20,0.0,4,euclidean,20,50,0.0,euclidean,30,0.724351,0.302626,0.421725,0.316
127,20,0.0,4,euclidean,20,50,0.1,euclidean,30,0.724351,0.302626,0.421725,0.317
107,10,0.0,8,euclidean,20,50,0.1,euclidean,28,0.719995,0.288649,0.431346,0.384
109,10,0.0,8,euclidean,50,50,0.0,euclidean,28,0.719995,0.288649,0.431346,0.380
108,10,0.0,8,euclidean,20,50,0.0,euclidean,28,0.719995,0.288649,0.431346,0.389
110,10,0.0,8,euclidean,50,50,0.1,euclidean,28,0.719995,0.288649,0.431346,0.386
69,10,0.0,4,euclidean,20,50,0.1,euclidean,29,0.716013,0.264930,0.451082,0.358
68,10,0.0,4,euclidean,20,50,0.0,euclidean,29,0.716013,0.264930,0.451082,0.345


In [4]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import hdbscan
from itertools import product
from time import time
import umap
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# ================== 1. 读取数据（与原代码一致） ==================
with open("../data/fashion/handled/pca64_itm_emb_np.pkl", "rb") as f:
    item_emb = pickle.load(f)  # shape: [n_items, dim]
X = np.asarray(item_emb)
X = StandardScaler().fit_transform(X)
print(f"数据形状: {X.shape}")

# # ================== 2. 细粒度参数网格（控制总组合数） ==================
# # UMAP 网格：聚焦 nn=10-20、nc=4-8 细化，md 保留 0.0/0.1（原结果中效果稳定）
# umap_grid = {
#     "n_neighbors": [10, 15, 20],    # 细化：原 10/20 加中间值 15
#     "min_dist": [0.0, 0.1],         # 保留原有效范围，不盲目加值
#     "n_components": [4, 6, 8],      # 细化：原 4/8 加中间值 6
#     "metric": ["euclidean"]         # 固定：原结果中无明显差异，减少冗余
# }

# # HDBSCAN 网格：聚焦 mcs=20-50、ms=20-50 细化，eps 保留 0.0/0.1
# hdb_grid = {
#     "min_cluster_size": [20, 30, 50],# 细化：原 20/50 加中间值 30
#     "min_samples": [20, 30, 50],    # 细化：原 20/50 加中间值 30
#     "cluster_selection_epsilon": [0.0, 0.1],  # 保留原范围
#     "metric": ["euclidean"]         # 固定：减少冗余
# }

# ================== 2. 更细粒度参数网格（基于Top5结果聚焦） ==================
# UMAP 网格：完全聚焦Top5高优区间（nn=8-12、nc=4-6、md=0.0-0.1）
umap_grid = {
    "n_neighbors": [8, 10, 12],     # 细化：Top5中nn=10最优，补充8/12验证邻近值
    "min_dist": [0.0, 0.05, 0.1],   # 细化：原0.0/0.1加中间值0.05，找临界点
    "n_components": [4, 5, 6],      # 细化：Top5中nc=4/6最优，加中间值5
    "metric": ["euclidean"]         # 固定：Top5中无差异，减少冗余
}

# HDBSCAN 网格：聚焦Top5高优区间（mcs=20-30、ms=20-30、eps=0.0-0.1）
hdb_grid = {
    "min_cluster_size": [20, 25, 30],# 细化：Top5中mcs=20/30最优，加中间值25
    "min_samples": [20, 25, 30],    # 细化：Top5中ms=20/30最优，加中间值25
    "cluster_selection_epsilon": [0.0, 0.05, 0.1],  # 细化：加0.05验证临界点
    "metric": ["euclidean"]         # 固定：减少冗余
}


# 计算总组合数（18 * 18 = 324 次，仅为原方案的 1/3）
umap_combos = list(product(*umap_grid.values()))
hdb_combos = list(product(*hdb_grid.values()))
total = len(umap_combos) * len(hdb_combos)
print(f"总搜索次数：{total}（细粒度且无冗余）")

rows = []
idx = 0

# ================== 3. 遍历组合（逻辑与原代码一致，仅参数网格变化） ==================
for u_params in umap_combos:
    nn, md, nc, met_r = u_params

    # UMAP 降维（复用原逻辑，固定 random_state 保证可复现）
    reducer = umap.UMAP(
        n_neighbors=nn, min_dist=md, n_components=nc,
        metric=met_r, random_state=42
    )
    Z = reducer.fit_transform(X)

    for h_params in hdb_combos:
        mcs, ms, eps, met_c = h_params
        idx += 1
        t0 = time()

        # HDBSCAN 聚类
        clusterer = hdbscan.HDBSCAN(
            min_cluster_size=mcs, min_samples=ms,
            cluster_selection_epsilon=eps, metric=met_c
        )
        labels = clusterer.fit_predict(Z)

        # ---- 评估指标（与原代码一致，确保指标可比） ----
        noise_ratio = float(np.mean(labels == -1))
        mask = labels != -1
        # 仅当有效聚类数≥2时计算轮廓系数（避免无意义值）
        if mask.sum() > 1 and len(np.unique(labels[mask])) >= 2:
            sil = float(silhouette_score(Z[mask], labels[mask]))
        else:
            sil = np.nan

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        # 打印进度（保留原格式，便于观察）
        print(f"[{idx}/{total}] UMAP(nn={nn}, md={md}, nc={nc}) + "
              f"HDBSCAN(mcs={mcs}, ms={ms}, eps={eps}) "
              f"=> Clusters={n_clusters}, Sil={sil:.4f}, Noise={noise_ratio:.3f}")

        # 记录结果
        rows.append({
            "nn": nn, "md": md, "nc": nc, "umap_metric": met_r,
            "mcs": mcs, "ms": ms, "eps": eps, "hdb_metric": met_c,
            "Clusters": n_clusters,
            "Silhouette": sil,
            "NoiseRatio": noise_ratio,
            "Score": (sil if not np.isnan(sil) else -1) - noise_ratio,  # 原评分逻辑
            "TimeSec": round(time() - t0, 3)
        })

# ================== 4. 保存 & 查看结果（与原代码一致） ==================
df = pd.DataFrame(rows)


数据形状: (4722, 64)
总搜索次数：729（细粒度且无冗余）


/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[1/729] UMAP(nn=8, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=20, eps=0.0) => Clusters=61, Sil=0.7241, Noise=0.206
[2/729] UMAP(nn=8, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=20, eps=0.05) => Clusters=61, Sil=0.7241, Noise=0.206
[3/729] UMAP(nn=8, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=20, eps=0.1) => Clusters=61, Sil=0.7241, Noise=0.206
[4/729] UMAP(nn=8, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=25, eps=0.0) => Clusters=51, Sil=0.7474, Noise=0.260
[5/729] UMAP(nn=8, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=25, eps=0.05) => Clusters=51, Sil=0.7474, Noise=0.260
[6/729] UMAP(nn=8, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=25, eps=0.1) => Clusters=51, Sil=0.7474, Noise=0.260
[7/729] UMAP(nn=8, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=30, eps=0.0) => Clusters=41, Sil=0.7193, Noise=0.263
[8/729] UMAP(nn=8, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=30, eps=0.05) => Clusters=41, Sil=0.7193, Noise=0.263
[9/729] UMAP(nn=8, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=30, eps=0.1) => Clusters=41, Sil=0.7193, Noise=0.263
[10/729] UMAP(nn=8, md=0.0, nc=4) 

/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[28/729] UMAP(nn=8, md=0.0, nc=5) + HDBSCAN(mcs=20, ms=20, eps=0.0) => Clusters=60, Sil=0.7281, Noise=0.198
[29/729] UMAP(nn=8, md=0.0, nc=5) + HDBSCAN(mcs=20, ms=20, eps=0.05) => Clusters=60, Sil=0.7281, Noise=0.198
[30/729] UMAP(nn=8, md=0.0, nc=5) + HDBSCAN(mcs=20, ms=20, eps=0.1) => Clusters=60, Sil=0.7281, Noise=0.198
[31/729] UMAP(nn=8, md=0.0, nc=5) + HDBSCAN(mcs=20, ms=25, eps=0.0) => Clusters=47, Sil=0.7201, Noise=0.219
[32/729] UMAP(nn=8, md=0.0, nc=5) + HDBSCAN(mcs=20, ms=25, eps=0.05) => Clusters=47, Sil=0.7201, Noise=0.219
[33/729] UMAP(nn=8, md=0.0, nc=5) + HDBSCAN(mcs=20, ms=25, eps=0.1) => Clusters=47, Sil=0.7201, Noise=0.219
[34/729] UMAP(nn=8, md=0.0, nc=5) + HDBSCAN(mcs=20, ms=30, eps=0.0) => Clusters=41, Sil=0.7090, Noise=0.251
[35/729] UMAP(nn=8, md=0.0, nc=5) + HDBSCAN(mcs=20, ms=30, eps=0.05) => Clusters=41, Sil=0.7090, Noise=0.251
[36/729] UMAP(nn=8, md=0.0, nc=5) + HDBSCAN(mcs=20, ms=30, eps=0.1) => Clusters=41, Sil=0.7090, Noise=0.251
[37/729] UMAP(nn=8, md=0.

...

[649/729] UMAP(nn=12, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=20, eps=0.0) => Clusters=53, Sil=0.6822, Noise=0.276
[650/729] UMAP(nn=12, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=20, eps=0.05) => Clusters=53, Sil=0.6822, Noise=0.276
[651/729] UMAP(nn=12, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=20, eps=0.1) => Clusters=53, Sil=0.6822, Noise=0.276
[652/729] UMAP(nn=12, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=25, eps=0.0) => Clusters=43, Sil=0.6694, Noise=0.298
[653/729] UMAP(nn=12, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=25, eps=0.05) => Clusters=43, Sil=0.6694, Noise=0.298
[654/729] UMAP(nn=12, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=25, eps=0.1) => Clusters=43, Sil=0.6694, Noise=0.298
[655/729] UMAP(nn=12, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=30, eps=0.0) => Clusters=37, Sil=0.6986, Noise=0.342
[656/729] UMAP(nn=12, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=30, eps=0.05) => Clusters=37, Sil=0.6986, Noise=0.342
[657/729] UMAP(nn=12, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=30, eps=0.1) => Clusters=37, Sil=0.6986, Noise=0.342
[658/72

/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[676/729] UMAP(nn=12, md=0.1, nc=5) + HDBSCAN(mcs=20, ms=20, eps=0.0) => Clusters=52, Sil=0.6800, Noise=0.264
[677/729] UMAP(nn=12, md=0.1, nc=5) + HDBSCAN(mcs=20, ms=20, eps=0.05) => Clusters=52, Sil=0.6800, Noise=0.264
[678/729] UMAP(nn=12, md=0.1, nc=5) + HDBSCAN(mcs=20, ms=20, eps=0.1) => Clusters=52, Sil=0.6800, Noise=0.264
[679/729] UMAP(nn=12, md=0.1, nc=5) + HDBSCAN(mcs=20, ms=25, eps=0.0) => Clusters=42, Sil=0.6835, Noise=0.297
[680/729] UMAP(nn=12, md=0.1, nc=5) + HDBSCAN(mcs=20, ms=25, eps=0.05) => Clusters=42, Sil=0.6835, Noise=0.297
[681/729] UMAP(nn=12, md=0.1, nc=5) + HDBSCAN(mcs=20, ms=25, eps=0.1) => Clusters=42, Sil=0.6835, Noise=0.297
[682/729] UMAP(nn=12, md=0.1, nc=5) + HDBSCAN(mcs=20, ms=30, eps=0.0) => Clusters=39, Sil=0.6771, Noise=0.297
[683/729] UMAP(nn=12, md=0.1, nc=5) + HDBSCAN(mcs=20, ms=30, eps=0.05) => Clusters=39, Sil=0.6771, Noise=0.297
[684/729] UMAP(nn=12, md=0.1, nc=5) + HDBSCAN(mcs=20, ms=30, eps=0.1) => Clusters=39, Sil=0.6771, Noise=0.297
[685/72

/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[703/729] UMAP(nn=12, md=0.1, nc=6) + HDBSCAN(mcs=20, ms=20, eps=0.0) => Clusters=50, Sil=0.6802, Noise=0.276
[704/729] UMAP(nn=12, md=0.1, nc=6) + HDBSCAN(mcs=20, ms=20, eps=0.05) => Clusters=50, Sil=0.6802, Noise=0.276
[705/729] UMAP(nn=12, md=0.1, nc=6) + HDBSCAN(mcs=20, ms=20, eps=0.1) => Clusters=50, Sil=0.6802, Noise=0.276
[706/729] UMAP(nn=12, md=0.1, nc=6) + HDBSCAN(mcs=20, ms=25, eps=0.0) => Clusters=39, Sil=0.6662, Noise=0.274
[707/729] UMAP(nn=12, md=0.1, nc=6) + HDBSCAN(mcs=20, ms=25, eps=0.05) => Clusters=39, Sil=0.6662, Noise=0.274
[708/729] UMAP(nn=12, md=0.1, nc=6) + HDBSCAN(mcs=20, ms=25, eps=0.1) => Clusters=39, Sil=0.6662, Noise=0.274
[709/729] UMAP(nn=12, md=0.1, nc=6) + HDBSCAN(mcs=20, ms=30, eps=0.0) => Clusters=37, Sil=0.6828, Noise=0.302
[710/729] UMAP(nn=12, md=0.1, nc=6) + HDBSCAN(mcs=20, ms=30, eps=0.05) => Clusters=37, Sil=0.6828, Noise=0.302
[711/729] UMAP(nn=12, md=0.1, nc=6) + HDBSCAN(mcs=20, ms=30, eps=0.1) => Clusters=37, Sil=0.6828, Noise=0.302
[712/72

In [5]:
df_sorted = df.sort_values(by="Silhouette", ascending=False)

print("\nTop 5 最优参数组合：")
print(df_sorted.head(5)[["nn", "md", "nc", "mcs", "ms", "eps", "Clusters", "Silhouette", "NoiseRatio", "Score"]])

# 保存结果（防止丢失，文件名保留 fashion 标识）
df_sorted.to_csv("umap_hdbscan_fine_results_fashion.csv", index=False)
print("\n细粒度搜索结果已保存到 umap_hdbscan_fine_results_fashion.csv")


Top 5 最优参数组合：
    nn   md  nc  mcs  ms   eps  Clusters  Silhouette  NoiseRatio     Score
4    8  0.0   4   20  25  0.05        51    0.747387    0.259848  0.487540
5    8  0.0   4   20  25  0.10        51    0.747387    0.259848  0.487540
3    8  0.0   4   20  25  0.00        51    0.747387    0.259848  0.487540
12   8  0.0   4   25  25  0.00        50    0.742579    0.251377  0.491202
14   8  0.0   4   25  25  0.10        50    0.742579    0.251377  0.491202

细粒度搜索结果已保存到 umap_hdbscan_fine_results_fashion.csv


In [7]:
df = pd.read_csv("umap_hdbscan_fine_results_fashion.csv")

In [8]:
df_filtered= df[
    (df["NoiseRatio"] >= 0.3)
]

In [9]:
df_filtered.head(20)

,nn,md,nc,umap_metric,mcs,ms,eps,hdb_metric,Clusters,Silhouette,NoiseRatio,Score,TimeSec
177,8,0.05,4,euclidean,25,30,0.00,euclidean,39,0.705238,0.301355,0.403883,0.329
178,8,0.05,4,euclidean,25,30,0.10,euclidean,39,0.705238,0.301355,0.403883,0.303
179,8,0.05,4,euclidean,20,30,0.05,euclidean,39,0.705238,0.301355,0.403883,0.318
180,8,0.05,4,euclidean,25,30,0.05,euclidean,39,0.705238,0.301355,0.403883,0.308
181,8,0.05,4,euclidean,20,30,0.10,euclidean,39,0.705238,0.301355,0.403883,0.308
182,8,0.05,4,euclidean,20,30,0.00,euclidean,39,0.705238,0.301355,0.403883,0.347
252,12,0.10,4,euclidean,20,30,0.10,euclidean,37,0.698553,0.342228,0.356325,0.283
253,12,0.10,4,euclidean,20,30,0.00,euclidean,37,0.698553,0.342228,0.356325,0.283
254,12,0.10,4,euclidean,20,30,0.05,euclidean,37,0.698553,0.342228,0.356325,0.281
339,10,0.10,5,euclidean,20,30,0.05,euclidean,37,0.694360,0.321474,0.372886,0.307
